# Similarity Modeling 1/2: Recognizing Pigs with Deep Learning

## Student
Gallus Huber (51905700)

## Timesheet
2022-11-24 16:00 - 17:30 Setup

## Required libraries and installs

In [ ]:
!cat requirements-dl-video.txt
#!pip install -r requirements-dl-video.txt

## Imports

In [ ]:
import os

import numpy as np
import pandas as pd

import cv2 as cv
import matplotlib.pyplot as plt

import sklearn.model_selection as ms

import keras.utils as utils
import keras.models as models
import keras.layers as layers

## Constants

In [ ]:
DATA_PATH = "../data/"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "sim12-ground-truth-muppets")
VIDEO_PATH = os.path.join(DATA_PATH, "video")

# 1. Data Preparation

## 1.1 Load ground truth

In [ ]:
ground_truth = []

for file_name in os.listdir(GROUND_TRUTH_PATH):
    if file_name.endswith(".csv"):
        csv = pd.read_csv(os.path.join(GROUND_TRUTH_PATH, file_name), sep=";")
        ground_truth.append(csv)

### 1.1.1 Plot ground truth data

In [ ]:
def plot_gt(gt, column):
    plot = plt.figure(figsize=(10, 4))

    # define subplot
    subplot = plot.add_subplot(111)
    subplot.set_title(f"Occurrences in Frames")

    # define x axis
    subplot.set_xlabel('Frames')

    # mark occurrences
    frame_indicator = np.zeros(len(gt['Frame_number']))
    frame_indicator[gt[gt[column] == 1].index] = 1

    subplot.plot(gt[column].index, frame_indicator)
    plt.show()

plot_gt(ground_truth[0], "Pigs")
plot_gt(ground_truth[1], "Pigs")
plot_gt(ground_truth[2], "Pigs")

## 1.2 Load video data
For training use the second video, as the pigs occur most often.

In [ ]:
second_video = "Muppets-02-04-04.avi"

def retrieve_frames(video_file):
    video_capture = cv.VideoCapture(os.path.join(VIDEO_PATH, video_file))
    
    video_capture.set(cv.CAP_PROP_FRAME_WIDTH, 224)
    video_capture.set(cv.CAP_PROP_FRAME_HEIGHT, 224)

    frames = []
    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            video_capture.release()
        frames.append(frame)

    return frames

video_frames = dict()
video_frames[second_video] = retrieve_frames(second_video)

print(f"Loaded frames of {second_video} successfully")

# 2 Model

## 2.1 One-Hot Encoding class

In [ ]:
one_hot_encoding_class = utils.to_categorical(ground_truth[1].Pigs)

## 2.2 Prepare sets

### 2.2.1 Remove last frame

In [ ]:
video_frames[second_video] = video_frames[second_video][ : -2]

### 2.2.2 Create train test split

In [ ]:
train_set, validation_set, train_set_gt, validation_set_gt = ms.train_test_split(video_frames[second_video], one_hot_encoding_class, test_size=0.4, random_state=44)

## 2.3 Build model

In [ ]:
model = models.Sequential()

model.add(layers.InputLayer((7*7*512,)))
model.add(layers.Dense(units=1024, activation='sigmoid'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()

## 2.5 Compile model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 2.6 Train model

In [ ]:
model.fit(train_set, train_set_gt, epochs=100, validation_data=(validation_set, validation_set_gt))

In [ ]:
model